# ⏱️ Zwei-Spieler – Restspieldauer bei synchroner Reaktion auf Würfe
In diesem Modell reagieren **beide Spieler gleichzeitig** auf ein gemeinsames Würfelergebnis.
Pro Runde wird **genau einmal gewürfelt**, und jeder Spieler entfernt (falls möglich) einen Chip vom gewürfelten Feld. Das Spiel endet, wenn:

- Spieler A **oder** Spieler B **keine Chips mehr** hat,
- oder **beide Spieler dieselbe Chipverteilung** haben.

Wir berechnen den Erwartungswert der verbleibenden Spielrunden: $E(U, V)$

## 🧮 Rekursive Formel für $E(U, V)$
Die rekursive Berechnung erfolgt nach folgender Formel:

$$
E(U, V) = 
\begin{cases}
0, & \text{falls } \sum U = 0 \text{ oder } \sum V = 0 \text{ oder } U = V \\
\frac{1}{s} + \sum\limits_{j=1}^m \frac{p_j}{s} \cdot  \mathbb{1}(u_j+v_j \ge 0) \cdot  E(U_j, V_j), & \text{sonst}
\end{cases}
$$

**Dabei gilt:**
- $s = \sum p_j \cdot \mathbb{1}(u_j+v_j \ge 0)$ ist die Gesamtwahrscheinlichkeit
- $U_j, V_j$ sind die Zustände nach Abziehen eines Chips (falls vorhanden) auf Feld $j$

## ⚙️ Setup

In [82]:
from fractions import Fraction
from functools import lru_cache

## 🎯 Wahrscheinlichkeiten und Startverteilungen

In [84]:
p = (Fraction(1, 2), Fraction(1, 3), Fraction(1, 6))
U = (4, 2, 0)
V = (3, 2, 1)

## 🔧 Hilfsfunktionen

In [86]:
def is_terminal(u, v):
    return sum(u) == 0 or sum(v) == 0 or u == v

def next_state_pair(u, v, index):
    u_new = list(u)
    v_new = list(v)
    if u[index] > 0:
        u_new[index] -= 1
    if v[index] > 0:
        v_new[index] -= 1
    return tuple(u_new), tuple(v_new)

## 🔁 Erwartungswert-Berechnung E(U, V)

In [88]:
@lru_cache(None)
def E(U, V):
    if is_terminal(U, V):
        return Fraction(0)

    total = sum(p[i] for i in range(len(U)) if U[i] > 0 or V[i] > 0)
    res = Fraction(1, total)

    for i in range(len(U)):
        if U[i] > 0 or V[i] > 0:
            u_new, v_new = next_state_pair(U, V, i)
            res += (p[i] / total) * E(u_new, v_new)

    return res

## 📊 Beispielauswertung

In [90]:
ev = E(U, V)
print(f"E({U}, {V}) = {ev} ({float(ev):.5})")

E((4, 2, 0), (3, 2, 1)) = 8868079/1200000 (7.3901)


## 🔁 Vergleich: Feste Strategie U vs. alle möglichen Gegenspieler V

In [92]:
from itertools import combinations_with_replacement

def all_distributions(n, m):
    raw = combinations_with_replacement(range(m), n)
    for comb in raw:
        counts = [0] * m
        for i in comb:
            counts[i] += 1
        yield tuple(counts)

results = []
for V_test in all_distributions(sum(U), len(U)):
    ev = E(U, V_test)
    results.append((V_test, ev))

results.sort(key=lambda x: x[1], reverse=True)  # Längste Restdauer zuerst

print(f"Top 5 Strategien gegen {U} (längste Restspielzeit):")
for Vx, ev in results[:5]:
    print(f"E({U}, {Vx}) = {ev} ({float(ev):.5})")

Top 5 Strategien gegen (4, 2, 0) (längste Restspielzeit):
E((4, 2, 0), (0, 0, 6)) = 6201485/663552 (9.3459)
E((4, 2, 0), (0, 1, 5)) = 73772645/7962624 (9.2649)
E((4, 2, 0), (1, 0, 5)) = 73689893/7962624 (9.2545)
E((4, 2, 0), (1, 1, 4)) = 7511017/829440 (9.0555)
E((4, 2, 0), (0, 6, 0)) = 17686494/1953125 (9.0555)
